In [3]:
import uproot
import h5py
import numpy as np
import os
import awkward as awk
import heplot as hepl
import importlib
import glob
import sys
sys.path.append('..')

In [4]:
import src.selection as sele
import src.string_constants as stco
import src.generator as gene

# read samples

## data

In [ ]:
N = int(1e4)
path_ee = glob.glob(os.path.join(stco.in_dir_data_run2,'*.root'))
generator_ee = gene.sample_generator(path_ee, N, feature_names, selection_fun=sele.select_lightjets)

samples_concat_ee = None
i = 0

for sample_batch in generator_ee:

    if samples_concat_ee is None:
        samples_concat_ee = sample_batch
    else:
        samples_concat_ee = awk.concatenate([samples_concat_ee,sample_batch])

    i = i+1
    if i > 20:
        break

print(f'{len(samples_concat_ee)} di-electron samples read')


In [5]:
path_data = glob.glob(os.path.join(stco.in_dir_data_run2,'*.root'))

In [10]:
fname_dat = os.path.join(stco.in_dir_data_run2,os.listdir(stco.in_dir_data_run2)[3])

In [11]:
fname_dat

'/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/dataRun2_daod_ftag2/user.ltoffoli.data17_13T.periodAllYear.physics_Main.PhysCont.D_FTAG2.grp17_v01_p5981.GN2v01_Nom_output.root/user.ltoffoli.37402699._000004.output.root'

In [18]:
tree = uproot.open(fname_dat+':nominal')

TypeError: open() takes 1 positional argument but 2 were given

In [17]:
tree.show(name_width=30, typename_width=20)

name                           | typename             | interpretation                
-------------------------------+----------------------+-------------------------------
eventNumber                    | uint64_t             | AsDtype('>u8')
runNumber                      | uint32_t             | AsDtype('>u4')
mcChannelNumber                | uint32_t             | AsDtype('>u4')
mu                             | float                | AsDtype('>f4')
mu_actual                      | float                | AsDtype('>f4')
mu_original_xAOD               | float                | AsDtype('>f4')
mu_actual_original_xAOD        | float                | AsDtype('>f4')
backgroundFlags                | uint32_t             | AsDtype('>u4')
hasBadMuon                     | uint32_t             | AsDtype('>u4')
el_pt                          | std::vector<float>   | AsJagged(AsDtype('>f4'), he...
el_eta                         | std::vector<float>   | AsJagged(AsDtype('>f4'), he...
el_cl_eta    

In [20]:
for batch in uproot.iterate(stco.in_dir_data_run2+"/*.root:nominal",None,steps=100):
    print(batch)

[{eventNumber: 1471875199, runNumber: 330294, mcChannelNumber: 0, ...}, ...]
[{eventNumber: 1120391, runNumber: 326439, mcChannelNumber: 0, ...}, ..., {...}]
[{eventNumber: 1181432055, runNumber: 330294, mcChannelNumber: 0, ...}, ...]
[{eventNumber: 111735957, runNumber: 326468, mcChannelNumber: 0, ...}, ...]
[{eventNumber: 303546754, runNumber: 331082, mcChannelNumber: 0, ...}, ...]
[{eventNumber: 485783514, runNumber: 327761, mcChannelNumber: 0, ...}, ...]
[{eventNumber: 224264351, runNumber: 332896, mcChannelNumber: 0, ...}, ...]
[{eventNumber: 696477452, runNumber: 330166, mcChannelNumber: 0, ...}, ...]


KeyboardInterrupt: 